<a href="https://colab.research.google.com/github/JisnaP/Agentic_AI_Powered_Chatbot_for_Queries_Related_to_CLAT_2025/blob/main/Agentic_AI_powered_chatbot_for_CLAT_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [132]:
%pip install tavily-python

In [134]:
!pip install llama_index

In [135]:
import llama_index

In [136]:
import os
OPENAI_KEY =os.getenv("OPENAI_KEY")

In [137]:
os.environ["OPENAI_KEY"] = "***"

In [139]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", api_key=OPENAI_KEY)

In [157]:
os.environ["TVLY_API"]="****"

In [161]:
os.environ["OPENAI_KEY"]="****"

In [167]:
import os
import asyncio
from tavily import AsyncTavilyClient
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from llama_index.core.agent.workflow import FunctionAgent, AgentWorkflow
from llama_index.core.memory import ChatMemoryBuffer


TVLY_API = os.getenv("TVLY_API")
OPENAI_KEY = os.getenv("OPENAI_KEY")


if not TVLY_API:
    raise ValueError("Tavily API key not found. Please set the TVLY_API environment variable.")
if not OPENAI_KEY:
    raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")

async def search_web(query: str) -> str:
    """Search the web for information and return results with limited content."""
    try:
        client = AsyncTavilyClient(api_key=TVLY_API)
        results = await client.search(query, include_answer=True, include_raw_content=False)


        search_result = results.get('answer', '') + "\n\nSources:\n"


        for i, result in enumerate(results.get('results', [])[:3]):
            search_result += f"- {result.get('title')}: {result.get('url')}\n"
            # Limit content to ~200 chars per source
            if 'content' in result:
                search_result += f"  {result.get('content')[:200]}...\n"

        return search_result
    except Exception as e:
        return f"Error searching the web: {str(e)}"


llm = OpenAI(
    model="gpt-3.5-turbo",
    api_key=OPENAI_KEY,
    max_tokens=4096,  # Limit output tokens
    temperature=0.7
)

# Create the search tool
tavily_search_tool = FunctionTool.from_defaults(
    fn=search_web,
    name="web_search",
    description="Use this tool to search the web for recent information. Always use this tool when asked about current events, dates, or specific facts."
)

# Create a prompt
system_prompt = (
    "You are a helpful assistant with web search capability. "
    "Always use the web_search tool for factual questions. "
    "Keep responses concise and focused on the query. "
    "Summarize search results without including all raw data."
)

# Create memory
memory = ChatMemoryBuffer.from_defaults(token_limit=4000)

# Create agent
agent = FunctionAgent(
    tools=[tavily_search_tool],
    llm=llm,
    system_prompt=system_prompt,
    memory=memory,
    verbose=True
)

# Create workflow
workflow = AgentWorkflow(agents=[agent])

async def main():
    try:
        response = await workflow.run("Give me last year’s cut-off for NLSIU Bangalore.")
        print(response)
    except Exception as e:
        print(f"Error running workflow: {str(e)}")

# Run the async function
if __name__ == "__main__":
    asyncio.run(main())

The NLSIU Bangalore cut-off for last year was around 140 in the CLAT exam. This rank was required for admission, but exact figures may vary slightly.
